## CSCE 421 Homework 4
#### Due: 10/19/2020
#### Jeffrey Xu

**1. Comparing standard normalization to PCA-shpering on MNIST:**

Compare a run of then gradient descent steps using the multi-class Softmax cost over 50,000 random digits from the MNIST dataset (introduced in Example 7.10), a standard normalized version, and a PCA-sphered version of the data. For each run use the largest fixed steplength of the form $\alpha=10^{\gamma}$ for $\gamma$ an integer you find to produce the descent. Create a plot comparing the progress of each run in terms of the cost function and number of misclassifications. Additionally, make sure your initialization of each run is rather small, particularly the first run where you apply no normalization at all to the input as each raw input point of this dataset is large in magnitude. In the case of the raw data initializing at a point too far away from the origin can easily cause numerical overflow, producing **nan** or **inf** values, ruining the rest of the corresponding local optimization run. 

In [244]:
# Include imports
from autograd import numpy as np
from autograd import grad
from numpy.random import rand
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_openml

# Load data
data = load_digits(return_X_y=True)
X = np.array(data[0])
y = np.array(data[1])

# Standard normalized data
X_norm = np.array(normalize(X))

# PCA normalized data
pca = PCA(n_components=10)
pca.fit(X)
X_PCA = np.array(pca.transform(X))

The loss function we are using is the multi-class softmax cost which is shown below. $$g({\bf w_{0},...,w_{C-1}})=\dfrac{1}{P}\sum_{p=1}^{P}[\log(\sum_{j=0}^{C-1}e^{x_{p}^{T}w_{j}})-x_{p}^{T}w_{y_{p}}]$$Our model looks like the following. (Textbook, equation 7.31, page 190). 

$$model(x,W)=x^{T}W=[x^{T}w_{0},x^{T}w_{1},...,x^{T}w_{C-1}]$$

In [239]:
# Complete model
def model(w, x):
    return (np.matmul(x, w[1:,]) + w[0])

# Softmax function for matrices
def soft(w, x):
    all_evals = model(w, x)
    s = np.exp(all_evals)
    return np.log(np.sum(s, axis=1))

# Multi-class softmax loss function
def loss(w, x, y):
    # Compute model for all values
    all_evals = model(w, x)
    
    a = soft(w, x)
    b = all_evals[np.arange(len(a)), y]
    cost = np.mean(a - b)
    return cost

Our output value will be the arguement max of our model. This will select the most likely digit that the data could be. 

$$y=argmax(model(x,W))$$

Now we want to perform gradient descent on our data. We need to perform gradient descent on the standard normalized data and the PCA-normalized data. 

**2. Image processing for human faces:**

In this problem, we will process face images coming from the Yale Face Dataset: **http://vision.ucsd.edu/content/yale-face-database**. This dataset contains images of the faces of 15 individuals. For each individual there are 11 images taken under a variety of conditions e.g., the person makes a happy expression, wears glasses, etc.

**(a)** Download the dataset from the above URL. *Implement* Principal Component Analysis(PCA) on the input images. Assume that the input vector of PCA contains all rows of an image stacked one on top of the other. You can use available libraries that calculate eigenvalues and eigenvectors of a matrix. *Hint:* Don't forget to normalize the data. 

**(b)** Plot a curve displaying the first $k$ eigenvalues $\lambda_{1},...,\lambda_{k}$, i.e. the energy of the first $K$ principal components. How many components do we need to capture 50% of the energy?

**(c)** Plot the top 10 *eigenfaces*, i.e. the eigenvectors **u_{k}**, $k=1,...,10$ obtained by PCA. 

**(d)** Select a couple of images from the data. Use the first $k$ eigenfaces as a basis to recontrust the images. Visualize the reconstructed images using 1, 10, 20, 30, 40, 50 components. How many components do we need to achieve a visually good result? *Hint:* Reconstruction of an input vector **x** based on the eigenvectors $u_{1},...,u_{k}$ is given by the following expression $x\approx x_{0}+\sum_{k=1}^{K}$, where $c_{k}=u_{k}^{T}x$ is the projection of the input image to the $k^{th}$ eigenvector. 

**(e)** *Perform face recognition:* Split the input data into training and testing *making sure that every person is included in each set.* Use as input features the transformed feature space that resulted from PCA. Experiment  with different number of PCA components through a 5-fold cross-validation. Use an outer 5-fold cross-validation to build predictors using support vector machines (using a radial basis function kernel) and logistic regression with lasso regularization. Report the recognition accuracy on the test set.  